In [1]:
import os

from tqdm import tqdm

from bs4 import BeautifulSoup
from lxml import etree

import json

import re
import html as html_lib

import pandas as pd

import pyperclip

In [2]:
directory_in = './file/2.part'
list_dict, list_dict_2, list_file = [], [], [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store', '0.null.txt'] and os.path.isfile(f'{directory_in}/{file}')]
try:
    crawler_status = True
    for file in tqdm(list_file, desc='Progress', ncols=77):
        with open(f'{directory_in}/{file}', 'r', encoding='UTF-8') as resp:
            text = resp.read()

        # = = = = = = = = = = = = = = =

        soup = BeautifulSoup(text, 'lxml')
        html = etree.HTML(str(soup))

        # = = = = = = = = = = = = = = =

        dict_, dict_2 = json.loads(text.split('<!-- Product Structured Data by Seosuite SEO-->')[1].split('<script type="application/ld+json">')[1].split('</script>')[0]), json.loads('{"[data-gallery-role=gallery-placeholder' + text.split('data-gallery-role=gallery-placeholder')[1].split('</script>')[0])

        # = = = = = = = = = = = = = = =

        dict_src, list_dict_src = {}, dict_2['[data-gallery-role=gallery-placeholder]']['mage/gallery/gallery']['data']
        for i, dict_src_temp in enumerate(list_dict_src):
            dict_src[str(i)] = re.sub(r'/cache/.*?/', '/', dict_src_temp['full']).strip()

        # = = = = = = = = = = = = = = =

        attribute, i, list_li = '', 0, html.xpath('//ul[@class="main_details_pr"]/li')
        for li in list_li:
            if li.xpath('./text()'):
                attribute += f'{i + 1}. {li.xpath('./text()')[0].strip()}\n'
                i += 1
            else:
                attribute_temp = etree.tostring(li).decode('utf-8')
                attribute_temp = re.sub(r'<.*?>', '', attribute_temp)
                attribute_temp = re.sub(r' +', ' ', attribute_temp.strip())
                attribute += f'\n{html_lib.unescape(attribute_temp)}\n'
                i = 0
            if li.xpath('./ul'):
                list_text = li.xpath('./ul/li/text()')
                for j, text in enumerate(list_text):
                    attribute += f'    {j + 1}. {text.strip()}\n'

        # = = = = = = = = = = = = = = =

        detail = etree.tostring(html.xpath('//div[@itemprop="description"]')[0]).decode('utf-8') if html.xpath('//div[@itemprop="description"]') else ''
        detail = re.sub(r'<.*?>', '', detail)
        detail = re.sub(r' +', ' ', detail.strip())
        detail = html_lib.unescape(detail)

        # = = = = = = = = = = = = = = =

        list_tr, dict_count = html.xpath('//table[@id="product-attribute-specs-table"]/tbody/tr'), {}
        for tr in list_tr:
            key = tr.xpath('./th/text()')[0].strip().title()
            if key in dict_count:
                dict_count[key] += 1
            else:
                dict_count[key] = 1

        dict_description, dict_no = {}, {}
        for i, tr in enumerate(list_tr):
            key = tr.xpath('./th/text()')[0].strip().title()
            value = etree.tostring(tr.xpath('./td')[0]).decode('utf-8')
            value = value.replace('<br/>', ';')
            value = re.sub(r'<.*?>', '', value)
            value = re.sub(r' +', ' ', value.strip())
            value = html_lib.unescape(value)
            if dict_count[key] == 1:
                dict_description[str(i)] = {key: value}
            else:
                if key in dict_no:
                    dict_no[key] += 1
                else:
                    dict_no[key] = 1
                dict_description[str(i)] = {f'{key}-{dict_no[key]}': value}

        # = = = = = = = = = = = = = = =

        list_a = html.xpath('//table[@id="product-attribute-specs-table_v2"]/tbody/tr/td/a')

        # = = = = = = = = = = = = = = =

        list_href = [a.xpath('./@href')[0].strip() for a in list_a]
        list_list_compatibility = [href.split('/catalog/')[1].split('/') for href in list_href]

        # = = = = = = = = = = = = = = =

        list_dict.append({'No': int(file.removesuffix('.html')),
                          'SKU': dict_['sku'].strip(),
                          'Brand': dict_['brand']['name'].strip(),
                          'MPN': dict_['mpn'].strip(),
                          'Title': dict_['name'].strip(),
                          'Currency': dict_['offers']['priceCurrency'].strip(),
                          'Price': dict_['offers']['price'],
                          'Picture': '',
                          'Url': html.xpath('//meta[@property="og:url"]/@content')[0].strip(),
                          'Attribute': attribute.strip(),
                          'Detail': detail,
                          'Compatibility': '\n'.join([a.xpath('./text()')[0].strip() for a in list_a]),
                          'Json_Src': json.dumps(dict_src),
                          'Json_Description': json.dumps(dict_description)})

        # = = = = = = = = = = = = = = =

        for list_compatibility in list_list_compatibility:
            list_dict_2.append({'SKU': dict_['sku'].strip(),
                                'Make': list_compatibility[0].strip().title(),
                                'Model': list_compatibility[1].strip().title(),
                                'Year': int(list_compatibility[2].strip())})
except:
    crawler_status = False
    print(file)
    print('fuck')

# = = = = = = = = = = = = = = =

if crawler_status:
    df_correct = pd.DataFrame(list_dict).sort_values(by=['No'],
                                                     ascending=[True],
                                                     ignore_index=True)
    df_correct.to_excel('./file/2.part.xlsx', index=False)

    # = = = = = = = = = = = = = = =

    df_correct = pd.DataFrame(list_dict_2).sort_values(by=['SKU', 'Make', 'Model', 'Year'],
                                                       ascending=[True, True, True, False],
                                                       ignore_index=True)
    df_correct.to_excel('./file/2.compatibility.xlsx', index=False)

df_correct

Progress: 100%|██████████████████████████████| 10/10 [00:00<00:00, 24.78it/s]


,SKU,Make,Model,Year
0,330-2052P-AC,Ford,F-150,2020
1,330-2052P-AC,Ford,F-150,2019
2,330-2052P-AC,Ford,F-150,2018
3,CS125-B000R,Dodge,Dakota,2000
4,CS125-B000R,Dodge,Dakota,2000
...,...,...,...,...
274,GM512-B000L,Buick,Regal,1997
275,LF030,Jeep,Wrangler,2017
276,LF030,Jeep,Wrangler,2016
277,LF030,Jeep,Wrangler,2015
